In [40]:
# Libraries used
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

In [99]:
# Importing data from mongoDB
def imp_from_mongodb(year):
    client = MongoClient ('localhost', 27017)
    db = client['companies']
    cur = db.companies.find({'founded_year': {'$gt': year }},
                                {'name':1, 'founded_year': 1, '_id': 0, 'number_of_employees':1,
                                 'offices.latitude':1, 'offices.longitude':1, 'funding_rounds.raised_amount':1, 
                                 'funding_rounds.raised_currency_code':1}) 
    comp_data = pd.DataFrame(cur)
    return comp_data

# Get latitude 
def get_lat(coord): 
    try: 
        return coord[0]['latitude']
    except:
        return None

# Get longitude    
def get_long(coord): 
    try: 
        return coord[0]['longitude']
    except:
        return None

# Inserting new columns with latitude and longitude    
def insert_latlong(comp_d):   
    comp_d['latitude'] = comp_d['offices'].apply(get_lat)
    comp_d['longitude'] = comp_d['offices'].apply(get_long)
    return comp_d

# Get funding amount
def get_raised(funding): 
    try: 
        return funding[0]['raised_amount']
    except:
        return None

# Get currency    
def get_currency(funding): 
    try: 
        return funding[0]['raised_currency_code']
    except:
        return None

# Inserting new columns with funding and currency    
def insert_funding(funding):   
    funding['raised_amount'] = funding['funding_rounds'].apply(get_raised)
    funding['raised_currency_code'] = funding['funding_rounds'].apply(get_currency)
    return funding

# Dropping columns with dictionarys in them
def col_drop(data):
    data = data.drop(['offices', 'funding_rounds'], axis=1)
    return data

In [100]:
# Main Code

companies_data = imp_from_mongodb(2005)
data_location = insert_latlong(companies_data)
data_loc_fund = insert_funding(data_location)
clean_data = col_drop(data_loc_fund)
clean_data.head()

,founded_year,name,number_of_employees,latitude,longitude,raised_amount,raised_currency_code
0,2006,Geni,18.0,34.090368,-118.393064,1500000.0,USD
1,2006,Slacker,NaN,33.022176,-117.081406,40000000.0,USD
2,2007,Scribd,50.0,37.789634,-122.404052,12000.0,USD
3,2007,MeetMoi,15.0,40.757929,-73.985506,1500000.0,USD
4,2007,Babelgum,NaN,53.344104,-6.267494,13200000.0,USD
